# Análisis comportamiento de clientes megaline

## Impotar las librerias que se van a utilizar

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df=pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Segmentar los datos en conjunto de entrenamiento, validación y prueba

Se divide el conjunto de datos en:

60% de entrenamiento

20% validación

20% prueba

In [4]:
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345)
df_valid,df_test=train_test_split(df_valid_test, test_size=0.5, random_state=12345)

In [5]:
#Se separa cada conjunto de datos en caracteristicas y el objetivo
features_train= df_train.drop('is_ultra',axis=1)
target_train=df_train['is_ultra']

features_valid=df_valid.drop('is_ultra',axis=1)
target_valid=df_valid['is_ultra']

features_test=df_test.drop('is_ultra',axis=1)
target_test=df_test['is_ultra']

##  Investigar los diferentes modelos

### Árbol de decisión

In [6]:
best_accuracy=0
best_depth=0
for depth in range(1,11):
    tree_model=DecisionTreeClassifier(max_depth=depth, random_state=12345)
    tree_model.fit(features_train,target_train)
    predictions=tree_model.predict(features_valid)
    accuracy=accuracy_score(target_valid,predictions)
    if accuracy>best_accuracy:
        best_accuracy=accuracy
        best_depth=depth
print (f'La mejor exactitud fue de {best_accuracy}, se obtuvo para un model con max_depth= {best_depth} ')

La mejor exactitud fue de 0.7853810264385692, se obtuvo para un model con max_depth= 3 


### Algoritmo bosques aleatorios

In [7]:
# Revisa si no eliminar
best_score = 0
best_est = 0
for est in range(1,40):
    forest_model = RandomForestClassifier(random_state=54321, n_estimators=est,max_depth=3) 
    forest_model.fit(features_train,target_train) 
    score = forest_model.score(features_valid,target_valid) 
    if score > best_score:
        best_score =score 
        best_est = est

print(f'La mejor exactitud fue de {best_accuracy}, se obtuvo para un model con n_estimators= {best_est} ')

La mejor exactitud fue de 0.7853810264385692, se obtuvo para un model con n_estimators= 33 


In [8]:
best_score = 0
best_est = 0
best_depth=0
for est in range(1,40):
    for depth in range(1,11):
        forest_model = RandomForestClassifier(random_state=54321, n_estimators=est, max_depth=depth) 
        forest_model.fit(features_train,target_train) 
        score = forest_model.score(features_valid,target_valid)
        score_train=forest_model.score(features_train,target_train)
        if score > best_score:
            best_score =score 
            best_est = est
            final_score_train=score_train
            best_depth=depth

print(f'La mejor exactitud fue de {best_accuracy}, se obtuvo para un model con n_estimators= {best_est} y max_depth= {best_depth}')
print(final_score_train)

La mejor exactitud fue de 0.7853810264385692, se obtuvo para un model con n_estimators= 33 y max_depth= 7
0.8609958506224067


### Regresión lógistica

In [9]:
regression_model=LogisticRegression(random_state=54321,solver='liblinear')
regression_model.fit(features_train,target_train)
score_valid=regression_model.score(features_valid,target_valid)
print(f' La exactitud para el modelo de regresión logistica es {score_valid}')

 La exactitud para el modelo de regresión logistica es 0.7589424572317263


De los 3 modelos se observa que el que tiene un mejor resultado es el algoritmo de bosques aleatorios, con un hiperparametro de n_estimators de 33. Se va a reentrenar este modelo con los datos combinados de el entrenamiento y validación 

## Comprueba la calidad del modelo usando el conjunto de prueba.

In [10]:
# juntar los datos de entrenamiento y validación
features= pd.concat([features_train,features_valid]).reset_index(drop=True)
target=pd.concat([target_train,target_valid]).reset_index(drop=True)

In [11]:
model=RandomForestClassifier(random_state=54321, n_estimators=33)
model.fit(features,target)
score_valid=model.score(features_test,target_test)
print(f' La exactitud para el modelo es {score_valid}')

 La exactitud para el modelo es 0.7900466562986003


## Hacer prueba de cordura al modelo

Para obtener más información sobre el rendimiento del modelo, se examinaran las métricas de precisión, recall y la puntuacion F1

Precisión es el número de miembros identificados correctamente de una clase dividido por todas las veces que el modelo predijo esa clase. 

Recall: la cantidad de miembros de una clase que el modelo haya indentificado correctamente dividido por el numero total de miembros de esa clase

F1 score: esta combina precisión y recall en una sola métrica. Si la precisión y recall son altas, f1 también lo es. Si ambos están bajos, F1 sera bajo. Si uno está alto y el otro bajo, F1 será bajo. 

In [12]:
#Se observa el reporte de clasificacion para la data de prueba y predicción
target_predict=model.predict(features_test)
print(classification_report(target_test, target_predict))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85       440
           1       0.70      0.58      0.63       203

    accuracy                           0.79       643
   macro avg       0.76      0.73      0.74       643
weighted avg       0.78      0.79      0.78       643



Al observar las metricas para la clase 1 (ultra), se observa que la presición es relativamente alta, lo que significa que el modelo evito etiquetar cosas como ultra que no lo son. Por otro lado recall es baja, lo que significa que el clasficador le falta una buena cantidad de ultra debido a que es demasiado cuidadoso. Y F1 score refleja este desequilibrio.

En cambio la metrica para la clase 0 (smart), la presición y recall son altos, lo que significa que el modelo no etiqueto muchos casos como smart que no lo son, y tampoco le falto clasificar muchos casos como smart cuando lo son. 